In [1]:
import json
import git
from git import Repo


f = open("sstubs.json")
sstubs = json.load(f)


filtered_bug_fixing_commits = {}
for bug in sstubs:
    if bug['projectName'] != "spring-projects.spring-boot":
        continue
    commit_id = bug["fixCommitSHA1"]
    parent_commit = bug["fixCommitParentSHA1"]
    bug_file_path = bug["bugFilePath"]
    bug_line_num = bug["bugLineNum"]
    if commit_id in filtered_bug_fixing_commits:
        if bug_file_path in filtered_bug_fixing_commits[commit_id]["bugFilePath"]:
            if bug_line_num in filtered_bug_fixing_commits[commit_id]["bugFilePath"][bug_file_path]:
                continue
            else:
                filtered_bug_fixing_commits[commit_id]["bugFilePath"][bug_file_path].add(bug_line_num)
        else:
            filtered_bug_fixing_commits[commit_id]["bugFilePath"][bug_file_path] = {bug_line_num}
    else:
        filtered_bug_fixing_commits[commit_id] = {
            'fixCommitParentSHA1': parent_commit,
            'bugFilePath': {bug_file_path: {bug_line_num}}
        }
print(filtered_bug_fixing_commits)

{'e3c87cba0bbed00c739e01599e29a5621c9ef3b3': {'fixCommitParentSHA1': 'c44a6dee3d0bf523339972715f752e878e2b346a', 'bugFilePath': {'spring-boot-project/spring-boot-autoconfigure/src/main/java/org/springframework/boot/autoconfigure/rsocket/RSocketServerAutoConfiguration.java': {74, 107}}}, 'ec57f0c31f8c41db175aa7dcaab5089c22b2a8a8': {'fixCommitParentSHA1': 'c44a6dee3d0bf523339972715f752e878e2b346a', 'bugFilePath': {'spring-boot-project/spring-boot-autoconfigure/src/main/java/org/springframework/boot/autoconfigure/rsocket/RSocketServerAutoConfiguration.java': {74, 107}}}, '0187e5106dfc5034b8b1650083eab37181ff0de3': {'fixCommitParentSHA1': 'b69f9e9fdf52d231cc7b4a2438fca667af383bdb', 'bugFilePath': {'spring-boot-project/spring-boot-autoconfigure/src/test/java/org/springframework/boot/autoconfigure/liquibase/LiquibaseAutoConfigurationTests.java': {305, 315}}}, '1ec8e822a17c37826b0a5df3b79ca03b4b528f1c': {'fixCommitParentSHA1': 'a76aaabc0773106e73d25574922888a42c9aa5dc', 'bugFilePath': {'sprin

In [2]:
repo = Repo("/Users/air/Documents/CS846/spring-boot")

heads = repo.heads
print(heads)

# repo.git.checkout('master')
# blames = repo.blame('HEAD', 'spring-boot-actuator/src/main/java/org/springframework/boot/actuate/autoconfigure/HealthIndicatorAutoConfiguration.java')
# print(blames[0:11])


repo.git.checkout('master')

# commits = repo.commit
# print(commits)

fix_and_bug_introducing_pairs = {}
for key, value in filtered_bug_fixing_commits.items():
    parent_commit = value["fixCommitParentSHA1"]
    for path_name, line_nums in value["bugFilePath"].items():
        for line_num in line_nums:
            line = 1
            for commit, lines in repo.blame(parent_commit, path_name):
                if line <= line_num < (line + len(lines)):
                    if key in fix_and_bug_introducing_pairs:
                        if commit not in fix_and_bug_introducing_pairs[key]:
                            fix_and_bug_introducing_pairs[key].add(commit.hexsha)
                    else:
                        fix_and_bug_introducing_pairs[key] = {commit.hexsha}
#                     print(commit)
                line += len(lines)
print(fix_and_bug_introducing_pairs)

[<git.Head "refs/heads/master">]
{'e3c87cba0bbed00c739e01599e29a5621c9ef3b3': {'b7e092e821021b7c0dbd91eacfe369181bfa28b8'}, 'ec57f0c31f8c41db175aa7dcaab5089c22b2a8a8': {'b7e092e821021b7c0dbd91eacfe369181bfa28b8'}, '0187e5106dfc5034b8b1650083eab37181ff0de3': {'eaad22dd89dbabf9fdd896956e3276a9f180b9aa'}, '1ec8e822a17c37826b0a5df3b79ca03b4b528f1c': {'fb0fccf3c7e1b1290ef64a1d2c388d788a5a965e'}, 'b6754f6f20ef786da0ecb542fe8b14bc01b88d62': {'e184298c50a0089336516933e5a6ce11a60df383'}, 'b7ceb6548fc986277173ca79065914877f1cc007': {'c6c139d980a50a1b39e564d44a1df6d46f8ca11b', '68a3fbd7a0453b815966db4353405a0b5e2303d3'}, 'eb9527a17038c6a44b4e1afa1fd82eb195baacb8': {'c6c139d980a50a1b39e564d44a1df6d46f8ca11b', '68a3fbd7a0453b815966db4353405a0b5e2303d3'}, 'b7e092e821021b7c0dbd91eacfe369181bfa28b8': {'7857dd2d7230ab48c53e8648c23647e3b793b895'}, '4083c721f2ee2ce54c624488a49ed08f7c3b3c39': {'d5370e6852cb61e333369127a517474de2a7ec58'}, 'f4d9e1c650ee6988bcd1807aaf02b74e3779358b': {'c2d1cb2c701f3c0dd06a6d

In [3]:
data = []
for key, value in fix_and_bug_introducing_pairs.items():
    commit_ids = list(value)
    data.append({
        'bug_fixing_commit_id': key,
        'bug_inducing_commit_ids': commit_ids
    })
    
with open('fix_and_bug_introducing_pairs.json', 'w') as outfile:
    json.dump(data, outfile)

In [4]:
bug_fixing_commits = []
for key, value in filtered_bug_fixing_commits.items():
    bug_inducing_commit_ids = list(fix_and_bug_introducing_pairs[key])
    locations = {}
    bugFilePath = value["bugFilePath"]
    for path_name, line_nums in bugFilePath.items():
        locations[path_name] = {
            'count': len(list(line_nums)),
            'locations': list(line_nums)
        }
    bug_fixing_commits.append({
        'bug_fixing_commit_id': key,
        'fixCommitParentSHA1': value["fixCommitParentSHA1"],
        'bug_inducing_commit_ids': bug_inducing_commit_ids,
        'bugFilePath': locations
    })
    
with open('bug_fixing_commits.json', 'w') as outfile:
    json.dump(bug_fixing_commits, outfile)

In [5]:
ln = 156
tlc = 1
commits = set()
for commit, lines in repo.blame('69fcae6b44cb9443bd970f89f7215258d6092bcc', 'spring-boot-actuator/src/main/java/org/springframework/boot/actuate/autoconfigure/HealthIndicatorAutoConfiguration.java'):
    if tlc <= ln < (tlc + len(lines)):
        commits.add(commit.hexsha)
        print(commit)
        print(lines)
    tlc += len(lines)
print(commits)

b569918db1c33847ce1d1464d7d78f692f289e99
['\t@ConditionalOnEnablednHealthIndicator("db")']
{'b569918db1c33847ce1d1464d7d78f692f289e99'}
